<font color=blue size=8>Industrial Recommendation System<font>


# Introduction and Evaluation

* 推荐系统介绍
* 推荐系统评估
1. 评估指标
2. 评估方法
* 推荐系统实践
1. 如何解决冷启动问题
2. 工业界推荐系统架构
3. 学术界和工业界的区别


* 推荐系统：不需要用户提供明确地需求，通过分析用户的历史行为给用户的兴趣进行建模，从而主动给用户推荐能够满足他们兴趣和需求的信息

![fig2](figures/fig2.jpg)

why

推荐系统存在的前提：
* 信息过载
* 用户需求不明确

推荐系统的目标
* 高效连接用户和物品，发现长尾商品
* 留住用户和内容生产者，实现商业目标

# 推荐系统评估

![fig1](figures/FIG1.jpg)

常见评估指标

* 准确性
* 满意度
* 覆盖率
* 多样性
* 新颖性
* 惊喜度
* 信任度
* 实时性
* 鲁棒性
* 可扩展性
* 商业目标
* 用户留存

Explicit vs Tmplicit

![fig4](figures/fig4.jpg)

准确性（学术界）
* 评分预测
$$
RMSE=\sqrt{\frac{\sum_{u,i\in T}(|r_{ui}-\hat{r}_{ui})^2}{|T|}}
$$

$$
MAE = \frac{\sum_{u,i\in T}|r_{ui}-\hat{r}_{ui}|}{|T|}
$$

* topN推荐
$$
Recall = 

准确性（工业界）

![fig5](figures/fig5.jpg)

覆盖度




多样性&新颖性&惊喜性

* 多样性：推荐列表中两两物品的不相似性。（相似性如何度量）
* 新颖性：未曾关注的类别、作者；推荐结果的平均流行度
* 惊喜性：历史不相似（惊）但很满意（喜）

往往需要牺牲准确性
* 使用历史行为预测用户对某个物品的喜爱程度
* 系统过度强调实时性

<font color=blue size=4>Explotitation & Exploration</font>

* Exploitation: 选择现有可能的最佳方案
* Exploration: 选择现有不确定的一些方案，但未来可能会有高收益的方案

在做两类决策的过程中，不断更新对现有决策的不确定性的认识，优化长期的目标函数

**Multi-armed bandit problem**

Bandit算法-原理

* Epsilon-Greedy:以1-$\epsilon$的概率选取当前受益最大的臂，以$\epsilon$的概率随机选取一个臂
* Upper Confidence Bound:均值越大，标准值越小，被选中的概率会越来越大

$$
x_j(t)+\sqrt{\frac{2\ln t}{T_{j,t}}}
$$

* Thompson Sampling:每个臂维护一个beta(wins,lose)分布，每次用现有的beta分布产生一个随机数，选择随机数最大的臂

In [ ]:
import numpy as np
import matplotlib.pylab as plt
import math
number_of_bandits = 10
number_of_arms = 10
number_of_pulls = 10000
epsilon = 0.3
min_temp = 0.1
decay_rate = 0.999

def pick_arm(q_values,counts,strategy,success,failure):
    global epsilon
    if strategy=="random":
        return np.random.randint(0,len(q_values))
    
    if strategy=="greedy":
        best_arms_value = np.max(q_values)
        best_arms = np.argwhere(q_values==best_arms_valuees).flatten()
        return best_arms[np.randoma.randint(0.,len(best_arms))]
    
    if strategy=="egreedy" or strategy=="egreedy_decay":
        if strategy=="egreedy_decay":
            epsilon = max(epsilon*decay_rate,min_temp)
        if np.random.random()>epsilon:
            best_arms_value = np.max(q_values)
            best_arms = np.argwhere(q_values==best_arms_valuee).flatten()
            return best_arms[np.random.randint(0,len(best_arms))]
        else:
            return np.random.randint(0,len(q_values))
        
    if strategy=="ucb":
        total_counts = np.sum(counts)
        q_values_ucb = q_values + np.sqrt(np.reciprocal(counts+0.001)*2*math.log(total_counts+1.0))
        best_arms_value = np.max(q_values_ucb)
        best_arms = np.argwhere(q_values_ucb==best_arms_valuee).flatten()
        return best_arms[np.random.randint(0,len(best_arms))]
    
    if strategy=="thompson":
        sample_means = np.zeros(len(counts))
        for i in range(len(counts)):
            sample_means[i] = np.random.beta(success[i]+1,failure[i]+1)
        return np.argmax(sample_means)
            

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
for st in ["greedy","random","egreedy","egreedy_decay","ucb","thompson"]:
    best_arm_counts = np.zeros((number_of_bandits,number_of_pulls))
    
    for i in range(number_of_bandits):
        arm_means = np.random.rand(number_of_arms)
        best_arm = np.argmax(arm_means)
        
        q_values = np.zeros(number_of_arms)
        counts = np.zeros(number_of_arms)
        success = np.zeros(number_of_arms)
        failure = np.zeros(number_of_arms)
        
        for j in range(number_of_pulls):
            a = pick_arm(q_values,counts,st,success,failure)
            
            reward = np.random.binomial(1,rarm_means[a])
            counts[a] += 1.0
            q_values[a] += (reward-q_values[a])/counts[a]

<font size=4 color=blue>Bandit算法-应用</font>

* 兴趣探索
* 冷启动探索
* LinUCB: 加入特征信息。用User和Item的特征预估回报及其置信区间，选择置信区间上界最大的Item推荐，观察回报后更新线性关系的参数，以此达到试验学习的目的。
* COFIBA:bandit协同过滤

----> * 基于用户聚类挑选最佳的Item(相似用户集成决策的Bandit);

----> * 基于用户的反馈情况调整User和Item的聚类(协同过滤部分）

EE实践

* 兴趣扩展：相似话题，搭配推荐
* 人群算法：userCF,用户聚类
* Bandit算法
* graph waiking
* 平衡个性化推荐和热门推荐比例
* 随机丢弃用户行为历史
* 随机扰动模型参数

眼前的苟且 & 远方的田野

* 探索伤害用户体验，可能导致用户流失
* 探索带来的长期受益（留存率）评估周期长，KPI压力大
* 如何平衡实时兴趣和长期兴趣？
* 如何平衡短期产品体验和长期系统生态？
* 如何平衡大众口味和小众需求？
* 如何避免劣币趋势良币？